In [1]:
import pandas as pd

In [2]:
df_historical_data = pd.read_csv("fifa_worldcup_historical_data.csv")
df_missing_data = pd.read_csv("fifa_worldcup_missing_data.csv")

In [4]:
# Adding missing data to historical data
df_historical_data = pd.concat([df_historical_data, df_missing_data], ignore_index=True)
#dropping potenital duplicates
df_historical_data.drop_duplicates(inplace=True)
df_historical_data.sort_values('year', inplace=True)
df_historical_data

,home,score,away,year
0,France,4–1,Mexico,1930
1,Argentina,1–0,France,1930
2,Chile,3–0,Mexico,1930
3,Chile,1–0,France,1930
4,Argentina,6–3,Mexico,1930
...,...,...,...,...
921,Croatia,1–1 (a.e.t.),Brazil,2022
928,Argentina,3–3 (a.e.t.),France,2022
927,Croatia,2–1,Morocco,2022
926,France,2–0,Morocco,2022


In [5]:
# deleting match with walk over
delete_index = df_historical_data[df_historical_data['home'].str.contains('Sweden') &
                                  df_historical_data['away'].str.contains('Austria')].index

df_historical_data.drop(index=delete_index, inplace=True)

# cleanning score and home/away columns
df_historical_data['score'] = df_historical_data['score'].str.replace('[^\d–]', '', regex=True)
df_historical_data['home'] = df_historical_data['home'].str.strip() # clean blank spaces: Yugoslavia twice
df_historical_data['away'] = df_historical_data['away'].str.strip()

# splitting score columns into home and away goals and dropping score column
df_historical_data[['HomeGoals', 'AwayGoals']] = df_historical_data['score'].str.split('–', expand=True)
df_historical_data.drop('score', axis=1, inplace=True)

# renaming columns and changing data types
df_historical_data.rename(columns={'home': 'HomeTeam', 'away': 'AwayTeam', 
                                   'year':'Year'}, inplace=True)
df_historical_data = df_historical_data.astype({'HomeGoals': int, 'AwayGoals':int, 'Year': int})

# creating new column "totalgoals"
df_historical_data['TotalGoals'] = df_historical_data['HomeGoals'] + df_historical_data['AwayGoals']
df_historical_data

,HomeTeam,AwayTeam,Year,HomeGoals,AwayGoals,TotalGoals
0,France,Mexico,1930,4,1,5
1,Argentina,France,1930,1,0,1
2,Chile,Mexico,1930,3,0,3
3,Chile,France,1930,1,0,1
4,Argentina,Mexico,1930,6,3,9
...,...,...,...,...,...,...
921,Croatia,Brazil,2022,1,1,2
928,Argentina,France,2022,3,3,6
927,Croatia,Morocco,2022,2,1,3
926,France,Morocco,2022,2,0,2


In [7]:
df_historical_data.to_csv('clean_fifa_worldcup_matches.csv',index=False)